In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata
import torch
from transformers import Trainer, TrainingArguments

# Obtener el token desde los secretos
hf_token = userdata.get('HF_TOKEN')

# Nombre del modelo
model_name = "bigscience/bloom-560m"

# Verificar si la GPU está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando el dispositivo: {device}")

# Cargar el tokenizador y el modelo usando el token desde secrets
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata

# Obtener tu token de Hugging Face desde Google Colab
hf_token2 = userdata.get('EHF_TOKEN2')

# Nombre del modelo
model_name = "distilbert/distilgpt2"  # Puedes cambiarlo por otro modelo compatible

# Cargar el tokenizador y modelo con el token de Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token)

In [ ]:
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Clase TicTacToe para manejar el juego
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # 9 celdas vacías
        self.current_turn = 'X'  # 'X' empieza primero

    def _printBoard(self):
        for i in range(0, 9, 3):
            print(f"{self.board[i]} | {self.board[i+1]} | {self.board[i+2]}")
            if i < 6:
                print("---------")

    def _checkWin(self, mark):
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
            [0, 4, 8], [2, 4, 6]             # Diagonales
        ]
        for condition in win_conditions:
            if all(self.board[i] == mark for i in condition):
                return True
        return False

    def _getEmptyCells(self):
        return [i for i, cell in enumerate(self.board) if cell == ' ']

    def playMove(self, position, mark):
        if self.board[position] == ' ':
            self.board[position] = mark
            return True
        return False

    def isFull(self):
        return ' ' not in self.board

def aiMove(model, tokenizer, device, board, mark):
    # Convertir el estado del tablero en un formato de texto para la IA
    board_state = ''.join(board)
    prompt = f"El tablero de TicTacToe es: {board_state}. ¿Dónde debe colocar la ficha '{mark}'?"

    # Tokenizar la entrada
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generar la respuesta del modelo
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=50)

    # Decodificar la salida
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Respuesta de la IA:", response)

    # Interpretar la respuesta para obtener la jugada
    move = None
    for i in range(9):
        if f"{i}" in response:
            move = i
            break

    # Validar si la posición está dentro del rango permitido
    if move is not None and 0 <= move < 9 and board[move] == ' ':
        return move
    else:
        print("Movimiento inválido, eligiendo una posición aleatoria.")
        return random.choice([i for i in range(9) if board[i] == ' '])


# Función evaluadora
def evaluateMove(board, position, mark):
    if board[position] == ' ':
        return "Válida"
    return "Inválida"

from google.colab import userdata

# Inicialización de modelos
hf_token = userdata.get('HF_TOKEN')  # jugador 1
hf_token2 = userdata.get('HF_TOKEN2')  # jugador 2
ehf_token = userdata.get('EHF_TOKEN') # evaluador

model_name_player1 = "bigscience/bloom-560m"
model_name_player2 = "TinyLlama/TinyLlama_v1.1"
model_name_evaluator = "distilgpt2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelos
tokenizer_p1 = AutoTokenizer.from_pretrained(model_name_player1, token=hf_token)
model_p1 = AutoModelForCausalLM.from_pretrained(model_name_player1, token=hf_token).to(device)

tokenizer_p2 = AutoTokenizer.from_pretrained(model_name_player2, token=hf_token2)
model_p2 = AutoModelForCausalLM.from_pretrained(model_name_player2, token=hf_token2).to(device)

tokenizer_eval = AutoTokenizer.from_pretrained(model_name_evaluator, token=ehf_token)
model_eval = AutoModelForCausalLM.from_pretrained(model_name_evaluator, token=ehf_token).to(device)


# def evaluateMove(model, tokenizer, device, board, position, mark):
#     # Convertir el estado del tablero en un formato de texto para la IA
#     board_state = ''.join(board)
#     prompt = f"El tablero de TicTacToe es: {board_state}. ¿Cuál es la próxima jugada válida para '{mark}'?"

#     # Tokenizar la entrada
#     inputs = tokenizer(prompt, return_tensors="pt").to(device)

#     # Generar la respuesta del modelo
#     with torch.no_grad():
#         outputs = model.generate(inputs['input_ids'], max_length=50)

#     # Decodificar la salida
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print("Respuesta de la IA Evaluadora:", response)

#     # Interpretar la respuesta para determinar si la jugada es válida
#     if "válida" in response.lower():
#         return "Válida"
#     else:
#         return "Inválida"
def evaluateMove(model, tokenizer, device, board, position, mark):
    # Convertir el estado del tablero en un formato de texto para la IA
    board_state = ''.join(board)

    # Crear un prompt que pregunte si la jugada en la posición actual es válida
    prompt = f"El tablero de TicTacToe es: {board_state}. ¿Cuál es la próxima jugada válida para '{mark}'?"

    # Tokenizar la entrada
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generar la respuesta del modelo
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=50)

    # Decodificar la salida
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Respuesta de la IA Evaluadora:", response)

    # Interpretar la respuesta para determinar si la jugada es válida
    if "válida" in response.lower():
        return "Válida"
    else:
        return "Inválida"

# Simular partidas
game = TicTacToe()
n_games = 10
wins_p1, wins_p2 = 0, 0

for game_num in range(1, n_games + 1):
    print(f"\nPartida {game_num}:")
    game.board = [' ' for _ in range(9)]
    game.current_turn = 'X'

    while not game.isFull():
        if game.current_turn == 'X':
            print("Turno del jugador IA 1 (X):")
            move = aiMove(model_p1, tokenizer_p1, device, game.board, 'X')
            evaluation = evaluateMove(model_eval, tokenizer_eval, device, game.board, move, 'X')
            # print(f"Evaluación de la jugada: {evaluation}")
            if evaluation == "Válida" and game.playMove(move, 'X'):
                game._printBoard()
                if game._checkWin('X'):
                    print("¡Jugador 1 (X) ha ganado!")
                    wins_p1 += 1
                    break
            game.current_turn = 'O'

        elif game.current_turn == 'O':
            print("Turno del jugador IA 2 (O):")
            move = aiMove(model_p2, tokenizer_p2, device, game.board, 'O')
            evaluation = evaluateMove(model_eval, tokenizer_eval, device, game.board, move, 'O')
            # print(f"Evaluación de la jugada: {evaluation}")
            if evaluation == "Válida" and game.playMove(move, 'O'):
                game._printBoard()
                if game._checkWin('O'):
                    print("¡Jugador 2 (O) ha ganado!")
                    wins_p2 += 1
                    break
            game.current_turn = 'X'

# Mostrar estadísticas finales
print("\nEstadísticas:")
print(f"Jugador 1 (X) - Victorias: {wins_p1}")
print(f"Jugador 2 (O) - Victorias: {wins_p2}")
print(f"Porcentaje de victorias del Jugador 1: {wins_p1 / n_games * 100}%")
print(f"Porcentaje de victorias del Jugador 2: {wins_p2 / n_games * 100}%")